In [14]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/prometeo23-kaggle/train_absa.csv
/kaggle/input/prometeo23-kaggle/sample.csv
/kaggle/input/prometeo23-kaggle/test_absa.csv


In [15]:
train_data= pd.read_csv('/kaggle/input/prometeo23-kaggle/train_absa.csv')
train_data.head()

,text,aspect,label
0,can you check whether its cancelled completely?,cancelled,1
1,cannot rely on both milk delivery and grocery ...,Milk,0
2,"I get no notification, however the app is real...",notification,0
3,"Love this app, but would love it even more if ...",view,1
4,it does not let me load a clip on the scene,load,0


In [9]:
train_data.shape   #(3500,3)
train_data['label'].value_counts()

0    1466
1    1133
2     901
Name: label, dtype: int64

Clearly there is no class imbalance in the data.

CLEANING TRAIN DATA

In [16]:
import re
import string
def text_clean_1(text):
    text=text.lower()
    text= re.sub('\[.*?\]','',text)
    text= re.sub('[%s]' % re.escape(string.punctuation),'',text)
    text= re.sub('\w*\d\w*','',text)
    text=re.sub('\n','',text)
    return text
cleaned1= lambda x: text_clean_1(x)

train_data["cleaned_text"]=pd.DataFrame(train_data.text.apply(cleaned1))
train_data["cleaned_aspect"]=pd.DataFrame(train_data.aspect.apply(cleaned1))
train_data.head()

,text,aspect,label,cleaned_text,cleaned_aspect
0,can you check whether its cancelled completely?,cancelled,1,can you check whether its cancelled completely,cancelled
1,cannot rely on both milk delivery and grocery ...,Milk,0,cannot rely on both milk delivery and grocery ...,milk
2,"I get no notification, however the app is real...",notification,0,i get no notification however the app is reall...,notification
3,"Love this app, but would love it even more if ...",view,1,love this app but would love it even more if g...,view
4,it does not let me load a clip on the scene,load,0,it does not let me load a clip on the scene,load


In [17]:
train_data['cleaned_final'] = train_data['cleaned_text'].str.cat(train_data['cleaned_aspect'], sep=' ')
train_data.head()


,text,aspect,label,cleaned_text,cleaned_aspect,cleaned_final
0,can you check whether its cancelled completely?,cancelled,1,can you check whether its cancelled completely,cancelled,can you check whether its cancelled completely...
1,cannot rely on both milk delivery and grocery ...,Milk,0,cannot rely on both milk delivery and grocery ...,milk,cannot rely on both milk delivery and grocery ...
2,"I get no notification, however the app is real...",notification,0,i get no notification however the app is reall...,notification,i get no notification however the app is reall...
3,"Love this app, but would love it even more if ...",view,1,love this app but would love it even more if g...,view,love this app but would love it even more if g...
4,it does not let me load a clip on the scene,load,0,it does not let me load a clip on the scene,load,it does not let me load a clip on the scene load


MODEL TRAINING

In [19]:
from sklearn.ensemble import RandomForestClassifier, StackingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
X= train_data['cleaned_final']
y= train_data['label']

from sklearn.model_selection import train_test_split
X_train,X_val,y_train,y_val= train_test_split(X,y,test_size=0.1,random_state=10)


base_learner_1 = [
    ('svr', make_pipeline(TfidfVectorizer(),
                           XGBClassifier(random_state=42)))
 ]
base_learner_2 = [
                  ('rf1', AdaBoostClassifier()),
                  ('rf2', GaussianNB())
                    
]
layer_two = StackingClassifier(estimators=base_learner_2, final_estimator=LogisticRegression())

model = StackingClassifier(estimators=base_learner_1, final_estimator=layer_two)
model.fit(X_train, y_train)

StackingClassifier(estimators=[('svr',
                                Pipeline(steps=[('tfidfvectorizer',
                                                 TfidfVectorizer()),
                                                ('xgbclassifier',
                                                 XGBClassifier(base_score=None,
                                                               booster=None,
                                                               callbacks=None,
                                                               colsample_bylevel=None,
                                                               colsample_bynode=None,
                                                               colsample_bytree=None,
                                                               early_stopping_rounds=None,
                                                               enable_categorical=False,
                                                               eval_metric=None,
  

MODEL EVALUATION

In [20]:
predictions=model.predict(X_val)
from sklearn.metrics import classification_report
print(classification_report(predictions,y_val))

              precision    recall  f1-score   support

           0       0.71      0.69      0.70       140
           1       0.77      0.59      0.67       133
           2       0.52      0.77      0.62        77

    accuracy                           0.67       350
   macro avg       0.67      0.68      0.66       350
weighted avg       0.69      0.67      0.67       350



In [21]:
test_data=pd.read_csv('/kaggle/input/prometeo23-kaggle/test_absa.csv')
test_data.head()

,text,aspect
0,i created the webinar on powerpoint and i woul...,slides
1,prices r very high for non milk items,milk
2,potato quality very small n inside black,quality
3,i.e. a knowledge hub for several content typ.,knowledge
4,"the best app for managment tasks, notes and so...",mobile app


In [22]:
test_data["cleaned_text"]=pd.DataFrame(test_data['text'].apply(cleaned1))
test_data["cleaned_aspect"]=pd.DataFrame(test_data['aspect'].apply(cleaned1))
test_data['cleaned_final'] = test_data['cleaned_text'].str.cat(test_data['cleaned_aspect'], sep=' ')
test_data.head()

,text,aspect,cleaned_text,cleaned_aspect,cleaned_final
0,i created the webinar on powerpoint and i woul...,slides,i created the webinar on powerpoint and i woul...,slides,i created the webinar on powerpoint and i woul...
1,prices r very high for non milk items,milk,prices r very high for non milk items,milk,prices r very high for non milk items milk
2,potato quality very small n inside black,quality,potato quality very small n inside black,quality,potato quality very small n inside black quality
3,i.e. a knowledge hub for several content typ.,knowledge,ie a knowledge hub for several content typ,knowledge,ie a knowledge hub for several content typ kno...
4,"the best app for managment tasks, notes and so...",mobile app,the best app for managment tasks notes and so ...,mobile app,the best app for managment tasks notes and so ...


In [23]:
predictions= model.predict(test_data['cleaned_final'])  
print(predictions)

[1 0 0 1 2 2 0 1 2 1 2 2 0 1 1 2 1 2 0 0 2 0 0 1 1 0 0 0 1 0 0 1 0 0 0 0 2
 0 0 1 0 0 0 2 2 2 1 1 1 0 1 2 1 2 2 0 1 1 2 1 0 2 1 0 0 1 2 1 0 1 2 1 2 2
 0 0 2 1 2 2 0 0 0 1 0 1 2 0 0 0 1 0 1 2 2 0 0 0 2 0 0 1 0 0 0 1 1 0 2 2 1
 2 2 0 0 1 0 0 1 0 1 0 1 1 2 1 1 1 2 0 2 2 1 0 0 2 2 1 0 0 0 1 0 0 1 0 1 0
 0 0 0 2 0 0 1 1 0 1 0 2 0 1 0 0 2 1 0 0 2 0 0 0 2 0 2 1 1 1 1 0 0 1 0 1 1
 0 0 1 0 0 2 1 1 1 1 0 0 1 0 1 0 0 1 1 2 2 1 0 1 0 1 0 2 0 0 0 1 1 0 1 0 1
 0 1 0 1 1 0 2 1 2 0 0 1 0 0 2 1 0 0 2 0 1 1 1 1 0 1 0 1 0 1 0 0 0 0 1 0 0
 2 0 1 0 0 1 0 0 0 0 0 0 1 1 2 0 0 1 0 2 1 1 1 2 2 1 2 0 1 0 2 0 1 1 2 0 1
 0 0 0 2 1 1 0 1 0 0 1 2 1 0 0 1 2 0 1 2 0 0 0 1 1 1 2 1 0 2 1 2 0 0 1 1 1
 0 1 0 1 0 1 0 2 0 0 1 2 1 1 2 0 1 1 0 0 2 1 1 1 0 1 2 1 1 2 0 0 0 0 2 0 1
 0 0 0 1 1 2 0 1 0 2 2 0 1 1 1 0 0 2 2 0 0 0 1 1 1 0 2 1 0 0 2 1 0 0 0 0 1
 0 0 1 1 0 1 2 0 1 0 2 2 0 1 2 1 1 1 0 2 2 0 1 0 1 0 0 0 0 1 0 0 0 2 0 0 2
 2 0 2 0 2 0 1 0 2 2 1 1 0 0 0 2 1 1 1 0 0 2 0 0 1 2 1 2 0 1 0 1 0 1 1 1 2
 1 0 1 0 0 1 0 0 2 1 2 2 

In [24]:
sample_submission= pd.read_csv('/kaggle/input/prometeo23-kaggle/sample.csv')
sample_submission.head()


,Id,Predicted
0,0,0
1,1,1
2,2,2
3,3,0
4,4,0


In [25]:
my_submission = pd.DataFrame({'Id': test_data.index, 'Predicted': predictions})
my_submission.head()

,Id,Predicted
0,0,1
1,1,0
2,2,0
3,3,1
4,4,2


In [ ]:
my_submission.to_csv('submission5.csv', index=False)